In [1]:
END = 150

# likely not necessary
vitA_reqs = [1000, 1000, 1000, 1000, 1000]
vitD_reqs = [10, 10, 10, 5, 5]
vitE_reqs = [10, 10, 10, 10, 10]
vitK_reqs = [45]

MAX_MEALS = 3

In [2]:
class Meal:
    
    default_id = 0

    def __init__(self, calories: int, protein: int, carbs: int, fats: int, cholesterol: int, sugar: int, name=""):
        
        if name == "":
            self.name = str(Meal.default_id)
            Meal.default_id += 1
        else:
            self.name = name
            
        self.calories = calories
        self.protein = protein
        self.carbs = carbs
        self.fats = fats
        self.cholesterol = cholesterol
        self.sugar = sugar
        
        self.nutrients = locals()
        self.nutrients.pop('self')
        
    def __repr__(self):
        return self.name
        
    def getNutrients(self) -> dict:
        return self.nutrients
    
    def getNutrient(self, nutrient: str) -> int:
        return self.nutrients[nutrient]

In [3]:
m = Meal(10, 10, 10, 10, 10, 10)
m.getNutrients()
m.getNutrient('sugar')

10

In [4]:
class DailyDiet:
    
    def __init__(self, meals=[]):
        self.validateMeals(meals)
        
        self.meals = meals
        
        self.nutrients = self.calculateTotals(meals)
                    
    def validateMeals(self, meals):
        if len(meals) > MAX_MEALS:
            raise ValueError('Too many meals included ')
            
    def calculateTotals(self, meals=None):
        if meals == None:
            meals = self.meals
        nutrients = dict()
        for meal in meals:
            for nutrient, value in meal.getNutrients().items():
                if nutrient not in nutrients:
                    nutrients.update({nutrient:value})
                else:
                    nutrients[nutrient] += value
    
        return nutrients

In [5]:
class Eater:
    
    # age groups
    age_groups = [(11,14), (15,18), (19,24), (25,50), (51,END)]

    # things we want
    calorie_reqs = [1600, 1800, 2000, 1800, 1600]
    protein_reqs = [0.20*item for item in calorie_reqs]
    carb_reqs = [0.45*item for item in calorie_reqs]
    fat_reqs = [0.35*item for item in calorie_reqs]
    
    requirements = [calorie_reqs, protein_reqs, carb_reqs, fat_reqs]

    # things to avoid
    chol_limits = [170, 170, 170, 200, 200]
    sugar_limits = [25, 25, 25, 25, 25]
    
    limits = [chol_limits, sugar_limits]
    
    def __init__(self, age, sex, meals = []):
        self.age = age
        self.age_index = self.findAgeIndex(age)
        self.sex = sex
        self.diet = DailyDiet(meals)
        self.nutrients = self.diet.calculateTotals()
        self.baseline = self.getBaselineNutrients()
        
    def getBaselineNutrients(self):
        i = self.age_index
        baseline = {
            "calories": Eater.calorie_reqs[i],
            "protein": Eater.protein_reqs[i],
            "carbs": Eater.carb_reqs[i],
            "fats": Eater.fat_reqs[i],
            "cholesterol": Eater.chol_limits[i],
            "sugar": Eater.sugar_limits[i]
        }
        return baseline
        
    def findAgeIndex(self, age):
        age_groups = Eater.age_groups
        for i in range(len(age_groups)):
            age_group = age_groups[i]
            if self.age <= age_group[1] and self.age >= age_group[0]:
                return i
            
    def calculateTotals(self):
        return self.diet.calculateTotals()
    
    def calculateDeficiency(self):
        deficiency = {}
        for nutrient, value in self.baseline.items():
            deficiency.update({nutrient: value - self.nutrients[nutrient]})
        return deficiency
    
    def calculateRelativeDeficiency(self, nutrients=None, baseline=None):
        if nutrients == None:
            nutrients = self.nutrients
        if baseline == None:
            baseline = self.baseline
        deficiency = {}
        for nutrient, value in baseline.items():
            deficiency.update({nutrient: 1.00 - (value - nutrients[nutrient])/value})
        return deficiency
        
        
        


In [6]:
m1 = Meal(10, 10, 10, 10, 10, 10)
m2 = Meal(20, 20, 20, 20, 20, 30)
m3 = Meal(1, 2, 3, 4, 5, 6)
m4 = Meal(1000, 40, 200, 40, 250, 100)

meals = [m1, m2, m3]
person = Eater(89, 'M', meals)
print(person.calculateTotals())
print(person.calculateDeficiency())
print(person.calculateRelativeDeficiency())

{'calories': 31, 'protein': 32, 'carbs': 33, 'fats': 34, 'cholesterol': 35, 'sugar': 46, 'name': ''}
{'calories': 1569, 'protein': 288.0, 'carbs': 687.0, 'fats': 526.0, 'cholesterol': 165, 'sugar': -21}
{'calories': 0.01937500000000003, 'protein': 0.09999999999999998, 'carbs': 0.04583333333333328, 'fats': 0.06071428571428572, 'cholesterol': 0.17500000000000004, 'sugar': 1.8399999999999999}


In [26]:
class ScoringEngine:
    
    def __init__(self):
        pass
        
    def calculateNewMealValue(self, eater, new_meal):
        new_nutrients = new_meal.getNutrients()
        deficiency_nutrients = eater.calculateDeficiency()
        completion = eater.calculateRelativeDeficiency(new_nutrients, deficiency_nutrients)
        return completion
        
    def scoreMeal(self, eater, new_meal):
        coverage = self.calculateNewMealValue(eater, new_meal)
        score = coverage['calories']*0.5 + coverage['protein']*0.20 + coverage['carbs']*0.10 + coverage['fats']*0.10  + coverage['sugar']*0.05 # + coverage['cholesterol']*0.05
        return score
    
    def scoreMeals(self, eater, new_meals=[]):
        meal_list = sorted(new_meals, key=lambda x: self.scoreMeal(eater, x), reverse=True)
        Meal.default_id = 0
        return meal_list
        

In [8]:
se = ScoringEngine()
print(se.scoreMeals(person, [m1, m2, m3, m4]))

[4, 1, 3, 2]


In [9]:
import requests

In [161]:
content = requests.get("https://api.nal.usda.gov/fdc/v1/foods/list?dataType=Foundation&api_key=####&pageSize=200")

In [185]:

def padNutrients(n):
    nutrients = dict()
    desireds = ['calories', 'carbs', 'protein', 'sugar', 'fats']
    for desired in desireds:
        if desired not in n:
            nutrients.update({desired:0})
        else:
            nutrients.update({desired:n[desired]})
        
    return nutrients

element = content.json()

print(element)


nutrition_packages = []

for element in content.json():
    nutritions = {}
    for n in element["foodNutrients"]:
        norm = n['name'].upper()
        cal_check = (norm == 'ENERGY' and n['unitName'].upper() == 'KCAL')
        carb_check = ('CARBOHYDRATE' in norm)
        protein_check = ('PROTEIN' in norm)
        sugar_check = ('SUGAR' in norm)
        fat_check = ('LIPID' in norm or 'FAT' in norm)
        if cal_check:
            nutritions.update({'calories': n['amount']})
        elif carb_check:
            nutritions.update({'carbs': n['amount']})
        elif protein_check:
            nutritions.update({'protein': n['amount']})
        elif sugar_check:
            nutritions.update({'sugar': n['amount']})
        elif fat_check:
            nutritions.update({'fats': n['amount']})
    nutrition_packages.append((element['description'], nutritions, element['fdcId']))

for element in nutrition_packages:
    print(element[0], padNutrients(element[1]), element[2])

[{'fdcId': 1105897, 'description': 'Apples, fuji, with skin, raw', 'dataType': 'Foundation', 'publicationDate': '2020-10-30', 'ndbNumber': '9504', 'foodNutrients': [{'number': '958', 'name': 'Energy (Atwater Specific Factors)', 'amount': 58.0, 'unitName': 'KCAL', 'derivationCode': 'NC', 'derivationDescription': 'Calculated'}, {'number': '957', 'name': 'Energy (Atwater General Factors)', 'amount': 65.0, 'unitName': 'KCAL', 'derivationCode': 'NC', 'derivationDescription': 'Calculated'}, {'number': '212', 'name': 'Fructose', 'amount': 8.4, 'unitName': 'G', 'derivationCode': 'A', 'derivationDescription': 'Analytical'}, {'number': '213', 'name': 'Lactose', 'amount': 0.0, 'unitName': 'G', 'derivationCode': 'A', 'derivationDescription': 'Analytical'}, {'number': '291', 'name': 'Fiber, total dietary', 'amount': 2.1, 'unitName': 'G', 'derivationCode': 'A', 'derivationDescription': 'Analytical'}, {'number': '405', 'name': 'Riboflavin', 'amount': 0.068, 'unitName': 'MG', 'derivationCode': 'A', 'd

In [136]:
food_ids = ",".join([str(element['fdcId']) for element in content.json()[:20]])
final_url = f'https://api.nal.usda.gov/fdc/v1/foods?fdcIds={food_ids}&nutrients=1008,1003,1004,2039,1063&dataType=Foundation&api_key=####'
food_content = requests.get(final_url)

In [138]:
foods = food_content.json()

print(foods[0])

for element in foods[:10]:
    print(element['description'])
    print(element['foodNutrients'][2]['nutrient']['name'], element['foodNutrients'][2]['nutrient']['id'], element['foodNutrients'][2]['nutrient']['unitName'])
    print(element['foodNutrients'][5]['nutrient']['name'], element['foodNutrients'][5]['nutrient']['id'], element['foodNutrients'][5]['nutrient']['unitName'])
    print(element['foodNutrients'][6]['nutrient']['name'], element['foodNutrients'][6]['nutrient']['id'], element['foodNutrients'][6]['nutrient']['unitName'])
    print(element['foodNutrients'][8]['nutrient']['name'], element['foodNutrients'][8]['nutrient']['id'], element['foodNutrients'][8]['nutrient']['unitName'])
    print(element['foodNutrients'][11]['nutrient']['name'], element['foodNutrients'][11]['nutrient']['id'], element['foodNutrients'][11]['nutrient']['unitName'])

    print('\n\n\n')

{'fdcId': 1105897, 'description': 'Apples, fuji, with skin, raw', 'publicationDate': '10/30/2020', 'foodNutrients': [], 'dataType': 'Foundation', 'foodClass': 'FinalFood', 'scientificName': 'Malus domestica', 'inputFoods': [{'id': 103310, 'foodDescription': 'APPLES, FUJI', 'inputFood': {'fdcId': 1105825, 'description': 'APPLES, FUJI', 'publicationDate': '10/30/2020', 'foodAttributeTypes': [], 'foodClass': 'Composite', 'totalRefuse': 0, 'dataType': 'Sample', 'foodGroup': {'id': 9, 'code': '0900', 'description': 'Fruits and Fruit Juices'}}}, {'id': 103308, 'foodDescription': 'APPLES, FUJI', 'inputFood': {'fdcId': 1105797, 'description': 'APPLES, FUJI', 'publicationDate': '10/30/2020', 'foodAttributeTypes': [], 'foodClass': 'Composite', 'totalRefuse': 0, 'dataType': 'Sample', 'foodGroup': {'id': 9, 'code': '0900', 'description': 'Fruits and Fruit Juices'}}}, {'id': 103313, 'foodDescription': 'APPLES, FUJI', 'inputFood': {'fdcId': 1105868, 'description': 'APPLES, FUJI', 'publicationDate': 

IndexError: list index out of range

In [133]:
foods = food_content.json()

'''for element in foods[:10]:
    print(element['description'])
    print(element['foodNutrients'])
'''
element = foods[0]
print(element['description'])
print(element['foodNutrients'])

Apples, fuji, with skin, raw
[{'nutrient': {'id': 2045, 'number': '951', 'name': 'Proximates', 'rank': 50, 'unitName': 'g'}, 'type': 'FoodNutrient'}, {'type': 'FoodNutrient', 'nutrient': {'id': 1051, 'number': '255', 'name': 'Water', 'rank': 100, 'unitName': 'g'}, 'foodNutrientDerivation': {'id': 1, 'code': 'A', 'description': 'Analytical', 'foodNutrientSource': {'id': 1, 'code': '1', 'description': 'Analytical or derived from analytical'}}, 'id': 13338384, 'amount': 83.6, 'dataPoints': 8, 'max': 84.7, 'min': 82.0, 'median': 83.8, 'minYearAcquired': 2020, 'nutrientAnalysisDetails': [{'subSampleId': 1105796, 'nutrientId': 1051, 'amount': 82.0, 'labMethodTechnique': 'Vacuum oven', 'labMethodDescription': 'AOAC 934.06 mod', 'labMethodOriginalDescription': '934.06 (37.1.10) Moisture in Fruits, Vegetables, and their Products', 'labMethodLink': 'http://www.aoac.org/aoac_prod_imis/AOAC/AOAC_Member/PUBSCF/OMACF/OMAP_M.aspx', 'nutrientAcquisitionDetails': [{'sampleUnitId': 1105783, 'purchaseDat

In [198]:
fid = 746763
content = requests.get(f"https://api.nal.usda.gov/fdc/v1/food/{fid}?dataType=Foundation&api_key=####")
print(content.json())

{'fdcId': 746763, 'footnote': 'Source number reflects the actual number of samples analyzed for a nutrient. Repeat nutrient analyses may have been done on the same sample with the values shown.', 'description': 'Beef, short loin, t-bone steak, bone-in, separable lean only, trimmed to 1/8" fat, choice, cooked, grilled', 'publicationDate': '12/16/2019', 'foodNutrients': [{'nutrient': {'id': 2045, 'number': '951', 'name': 'Proximates', 'rank': 50, 'unitName': 'g'}, 'type': 'FoodNutrient'}, {'type': 'FoodNutrient', 'nutrient': {'id': 1051, 'number': '255', 'name': 'Water', 'rank': 100, 'unitName': 'g'}, 'foodNutrientDerivation': {'id': 1, 'code': 'A', 'description': 'Analytical', 'foodNutrientSource': {'id': 1, 'code': '1', 'description': 'Analytical or derived from analytical'}}, 'id': 8514338, 'amount': 60.3, 'dataPoints': 24, 'max': 64.1, 'min': 56.4, 'median': 60.0, 'minYearAcquired': 2011, 'nutrientAnalysisDetails': [{'subSampleId': 334966, 'nutrientId': 1051, 'amount': 58.7, 'labMeth

In [207]:
food = content.json()
nutritions = {}
for element in food["foodNutrients"]:
    n = element['nutrient']
    norm = n['name'].upper()
    cal_check = (norm == 'ENERGY' and n['unitName'].upper() == 'KCAL')
    carb_check = ('CARBOHYDRATE' in norm)
    protein_check = ('PROTEIN' in norm)
    sugar_check = ('SUGAR' in norm)
    fat_check = ('LIPID' in norm or 'FAT' in norm)
    if 'amount' in element:
        if cal_check:
            nutritions.update({'calories': element['amount']})
        elif carb_check:
            nutritions.update({'carbs': element['amount']})
        elif protein_check:
            nutritions.update({'protein': element['amount']})
        elif sugar_check:
            nutritions.update({'sugar': element['amount']})
        elif fat_check:
            nutritions.update({'fats': element['amount']})
            
print(nutritions)

{'calories': 219.0, 'protein': 27.3, 'fats': 0.06, 'carbs': 0.0}
